Variables de los DataSets
Survived: Survival (0 = No, 1 = Yes)
Pclass: Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
Sex: Sex
Age: Age in years
SibSp: # of siblings / spouses aboard the Titanic
Parch: # of parents / children aboard the Titanic
Ticket: Ticket number
Fare: Passenger fare (tarifa del pasajero)
Cabin: Cabin number
Embarked: Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)

In [90]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import warnings
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from lazypredict.Supervised import LazyRegressor, LazyClassifier
import pyforest
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental    import enable_hist_gradient_boosting
from sklearn.ensemble        import HistGradientBoostingRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold

In [91]:
train_data = pd.read_csv("train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [92]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.83,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.00,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.69,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.66,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.29,NaN,S


In [93]:
gender = pd.read_csv("gender_submission.csv")
gender.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [94]:
train_data.shape

(891, 12)

In [95]:
test_data.shape

(418, 11)

In [96]:
gender.shape

(418, 2)

In [97]:
temp_data = test_data.merge(gender, how="left", on="PassengerId")
temp_data.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.50,0,0,330911,7.83,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00,1,0,363272,7.00,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.00,0,0,240276,9.69,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.00,0,0,315154,8.66,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00,1,1,3101298,12.29,NaN,S,1


In [98]:
titanic = pd.merge(train_data, temp_data, how="outer")

titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [99]:
titanic.shape

(1309, 12)

In [100]:
titanic.to_csv('titanic.csv')

In [101]:
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.00,1309.00,1309.00,1046.00,1309.00,1309.00,1308.00
mean,655.00,0.38,2.29,29.88,0.50,0.39,33.30
std,378.02,0.48,0.84,14.41,1.04,0.87,51.76
min,1.00,0.00,1.00,0.17,0.00,0.00,0.00
25%,328.00,0.00,2.00,21.00,0.00,0.00,7.90
50%,655.00,0.00,3.00,28.00,0.00,0.00,14.45
75%,982.00,1.00,3.00,39.00,1.00,0.00,31.27
max,1309.00,1.00,3.00,80.00,8.00,9.00,512.33


In [102]:
titanic.isnull().sum()

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [103]:
titanic['Fare'].isna().sum()

1

In [104]:
titanic[titanic['Fare'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1043,1044,0,3,"Storey, Mr. Thomas",male,60.50,0,0,3701,NaN,NaN,S


In [105]:
#titanic.query('Pclass == 3 & Embarked == "S" & SibSp == 0 & Parch == 0')
titanic[(titanic['Pclass'] == 3) 
        & (titanic['Embarked'] == 'S') 
        & (titanic['SibSp'] == 0)
        & (titanic['Parch'] == 0)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S
12,13,0,3,"Saundercock, Mr. William Henry",male,20.00,0,0,A/5. 2151,8.05,NaN,S
14,15,0,3,"Vestrom, Miss. Hulda Amanda Adolfina",female,14.00,0,0,350406,7.85,NaN,S
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.90,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1289,1290,0,3,"Larsson-Rondberg, Mr. Edvard A",male,22.00,0,0,347065,7.78,NaN,S
1303,1304,1,3,"Henriksson, Miss. Jenny Lovisa",female,28.00,0,0,347086,7.78,NaN,S
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.05,NaN,S
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.50,0,0,SOTON/O.Q. 3101262,7.25,NaN,S


In [106]:
titanic['Cabin'].isna().sum()

1014

In [107]:
# eliminamos la columna de cabin, teniendo que tiene 1014 valores nulos de un total de 1309
titanic.drop(['Cabin'], axis=1, inplace= True)

In [108]:
titanic['Embarked'].isna().sum()

2

In [109]:
titanic[titanic['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.00,0,0,113572,80.00,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.00,0,0,113572,80.00,NaN


In [110]:
titanic[titanic['Pclass'] == 1]['Embarked'].mode()[0]

'S'

In [111]:
titanic.loc[titanic['Embarked'].isnull(), 'Embarked'] = titanic[titanic['Pclass'] == 1]['Embarked'].mode()[0]

In [112]:
titanic['Age'].isna().sum()

263

In [113]:
titanic[(titanic['Age'].isna())]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.00,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.22,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.22,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.88,Q
...,...,...,...,...,...,...,...,...,...,...,...
1299,1300,1,3,"Riordan, Miss. Johanna Hannah""""",female,NaN,0,0,334915,7.72,Q
1301,1302,1,3,"Naughton, Miss. Hannah",female,NaN,0,0,365237,7.75,Q
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.05,S
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.05,S


In [114]:
round((titanic['Age'].count() / titanic['PassengerId'].count()) * 100, 2)

79.91

In [115]:
titanic[(titanic['Age'].notna())].groupby(['Pclass', 'Sex'])['Age'].mean()

Pclass  Sex   
1       female   37.04
        male     41.03
2       female   27.50
        male     30.82
3       female   22.19
        male     25.96
Name: Age, dtype: float64

In [116]:
titanic['Age'] = titanic.groupby(['Pclass','Sex'])['Age'].transform(lambda x: x.fillna(x.mean()))

In [117]:
len(titanic)

1309

In [118]:
titanic.groupby(['Sex'])['PassengerId'].count()

Sex
female    466
male      843
Name: PassengerId, dtype: int64

In [119]:
print('Total sobrevivientes: ', len(titanic[titanic['Survived'] == 1]))
titanic[titanic['Survived'] == 1].groupby(['Sex'])['PassengerId'].count()

Total sobrevivientes:  494


Sex
female    385
male      109
Name: PassengerId, dtype: int64

In [120]:
total_mujeres = len(titanic[titanic['Sex'] == 'female'])
total_hombres = len(titanic[titanic['Sex'] == 'male'])
print('Total pasajeros mujeres: ', total_mujeres)
print('Total pasajeros hombres: ', total_hombres)

sobrevivientes_mujeres = len(titanic[(titanic['Sex'] == 'female') & (titanic['Survived'] == 1)])
sobrevivientes_hombres = len(titanic[(titanic['Sex'] == 'male') & (titanic['Survived'] == 1)])
print('Sobrevivientes mujeres: ', (sobrevivientes_mujeres / total_mujeres) * 100)
print('Sobrevivientes hombres: ', (sobrevivientes_hombres / total_hombres) * 100)

Total pasajeros mujeres:  466
Total pasajeros hombres:  843
Sobrevivientes mujeres:  82.61802575107296
Sobrevivientes hombres:  12.930011862396205


In [121]:
titanic.groupby(['Pclass'])[['PassengerId']].count()

,PassengerId
Pclass,
1,323
2,277
3,709


In [122]:
titanic[titanic['Survived'] == 1].groupby(['Sex', 'Pclass'])[['Survived']].count()

Survived
Sex    Pclass          
female 1            141
       2            100
       3            144
male   1             45
       2             17
       3             47

In [123]:
total_1 = len(titanic[titanic['Pclass'] == 1])
total_2 = len(titanic[titanic['Pclass'] == 2])
total_3 = len(titanic[titanic['Pclass'] == 3])
print('Total pasajeros primera clase: ', total_1)
print('Total pasajeros segunda clase: ', total_2)
print('Total pasajeros tercera clase: ', total_3)

sobrevivientes_1 = len(titanic[(titanic['Pclass'] == 1) & (titanic['Survived'] == 1)])
sobrevivientes_2 = len(titanic[(titanic['Pclass'] == 2) & (titanic['Survived'] == 1)])
sobrevivientes_3 = len(titanic[(titanic['Pclass'] == 3) & (titanic['Survived'] == 1)])
print('Sobrevivientes primera clase: ', (sobrevivientes_1 / total_1) * 100)
print('Sobrevivientes segunda clase: ', (sobrevivientes_2 / total_2) * 100)
print('Sobrevivientes tercera clase: ', (sobrevivientes_3 / total_3) * 100)

Total pasajeros primera clase:  323
Total pasajeros segunda clase:  277
Total pasajeros tercera clase:  709
Sobrevivientes primera clase:  57.585139318885446
Sobrevivientes segunda clase:  42.238267148014444
Sobrevivientes tercera clase:  26.939351198871652


In [124]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [125]:
titanic_def = titanic.merge(output), how="left", on="PassengerId")
titanic_def.head(5)

SyntaxError: cannot assign to function call (3409428593.py, line 1)